# Computing the forward model

`
Authors: Britta Westner
`

`
based on existing tutorials by Alex Gramfort, Eric Larson, and Denis A. Engemann
`


This tutorial is not as interactive as the others (no exercises). In the context of the CuttingEEGX MNE-Python workshop, it is meant as a resource to check out if you are curious about forward modelling - the workshop is however focused on inverse modelling (source reconstruction).


In [1]:
%matplotlib inline
import mne
import matplotlib.pyplot as plt

mne.set_log_level('warning')

# set paths:
sample_path = mne.datasets.sample.data_path()
data_path = sample_path / 'MEG' / 'sample'
subjects_dir = sample_path / 'subjects'

# path to raw MEG/EEG file:
fname_raw = data_path / 'sample_audvis_raw.fif'

# Computing the forward operator

To compute a forward operator (also known as forward model, gain matrix, or - less accurately - lead field) we need:

   - a ``-trans.fif`` file that contains the coregistration information
   - a source space
   - the BEM surfaces

## Compute and visualize BEM surfaces


Here, we work with **pre-computed BEM surfaces**. 

Computing the BEM surfaces requires FreeSurfer (get it at: https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall#Download) and makes use of either of the two following command line tools:

[mne watershed_bem](http://martinos.org/mne/dev/generated/commands.html#mne-watershed-bem)

[mne flash_bem](http://martinos.org/mne/dev/generated/commands.html#mne-flash-bem)

Or can be done by directly by calling the functions (FreeSurfer installation needed):

https://mne.tools/stable/generated/mne.bem.make_watershed_bem.html

https://mne.tools/stable/generated/mne.bem.make_flash_bem.html





If you would like to try out BEM reconstruction for yourself later, you can install FreeSurfer on MacOS or Linux directly and in a VirtualMachine for Windows. 

To compute BEM models for this dataset,  you set up a `SUBJECTS_DIR` and run:

    mne watershed_bem -s sample --overwrite
    mne make_scalp_surfaces -s sample --force --overwrite

So, let's first look at the BEM surfaces.

For EEG we use 3 layers (inner skull, outer skull, and skin), while for MEG 1 layer (inner skull) is enough.


In [2]:
# first set the path to the T1
t1_fname = subjects_dir / 'sample' / 'mri' / 'T1.mgz'

We can have a look at the MRI using `nilearn`:

In [ ]:
%matplotlib inline
from nilearn import plotting
plotting.plot_anat(t1_fname);
plt.show()

We can also look at the BEM model (the identified surfaces of the BEM):

In [4]:
%matplotlib qt
mne.viz.plot_bem(subject='sample', subjects_dir=subjects_dir,
                 mri=t1_fname,
                 orientation='coronal');

## Coregistration 

The next step usually would be to coregister the MRI coordinate system with the MEG coordinate system. This is done to get the sensors into the right relation to the head model for the forward model computation.

For this data set, this is already done, but if you wanted to do this, this is the code to use:

In [5]:
# mne.gui.coregistration(subject='sample', subjects_dir=subjects_dir, inst=fname_raw);

### Visualizing the coregistration

The coregistration is the operation that allows to position the head and the sensors in a common coordinate system. 

In the MNE software, the transformation to align the head and the sensors in stored in a so called *trans* file. It is a FIF file that ends with `-trans.fif`. It can be obtained with ``mne_analyze`` (Unix tools), ``mne.gui.coregistration`` (in Python, see above) or mrilab if you're using a Neuromag system.

For the Python version, see https://mne.tools/dev/generated/mne.gui.coregistration.html for the docstring. There is also a video link that shows how to perform coregistration.

Since we assume the coregistration is done, we just visually check the alignment with the following code.

In [ ]:
trans_fname = data_path / 'sample_audvis_raw-trans.fif'
info = mne.io.read_info(fname_raw)
fig = mne.viz.plot_alignment(info, trans_fname, subject='sample', dig=True,
                             subjects_dir=subjects_dir, verbose=True);

## Compute Source Space

The source space defines the positions of the candidate source locations. The following code computes such a source space with an OCT-6 resolution. 

Note that this is a surface (not volume) source space.

In [7]:
mne.set_log_level('WARNING')
subject = 'sample'
src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

In [ ]:
src

`src` contains two parts, one for the left hemisphere (4098 locations) and one for the right hemisphere (4098 locations).

We can visualize the source space together with the head model and also together with the MEG helmet.

In [ ]:
%matplotlib inline
mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=False, src=src,
                             subjects_dir=subjects_dir, verbose=True, meg=False,
                             eeg=False);

In [ ]:
mne.viz.plot_alignment(info, trans_fname, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

### Compute forward solution

Now we have all the ingredients to compute the forward solution.

To reduce computational load, we'll just compute a single layer BEM
(just inner skull) that can then be used for MEG (but not EEG).

First, we compute the BEM model using `mne.make_bem_solution()`, then we compute the forward solution using `mne.make_forward_solution()`.

In [11]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Now compute the forward model (this might take a while!):

In [12]:
fwd = mne.make_forward_solution(fname_raw, trans=trans_fname,
                                src=src, bem=bem,
                                meg=True,  # include MEG channels
                                eeg=False,  # exclude EEG channels
                                mindist=5.0,  # ignore sources <= 5mm from inner skull
                                n_jobs=1)  # number of jobs to run in parallel

In [ ]:
fwd

In [ ]:
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)